# Using formulas to specify models

Formulas can be used to specify models using mostly standard [formulaic](https://matthewwardrop.github.io/formulaic/) syntax. Since system estimation is more complicated than the specification of a single model, there are two methods available to specify a system:

* Dictionary of formulas
* Single formula separated using {}

These examples use data on fringe benefits from F. Vella (1993), "A Simple Estimator for Simultaneous Models with Censored
Endogenous Regressors" which appears in Wooldridge (2002).  The model consists of two equations, one for hourly wage and the other for hourly benefits.  The initial model uses the same regressors in both equations. 

In [1]:
import numpy as np
import pandas as pd
from linearmodels.datasets import fringe

data = fringe.load()

## Dictionary 

The dictionary syntax is virtually identical to standard [formulaic syntax](https://matthewwardrop.github.io/formulaic/) where each equation is specified in a key-value pair where the key is the equation label and the value is the formula. It is recommended to use an OrderedDict which will preserve equation order in results. Keys **must** be strings.

In [2]:
from collections import OrderedDict

formula = OrderedDict()
formula[
    "benefits"
] = "hrbens ~ educ + exper + expersq + union + south + nrtheast + nrthcen + male"
formula["earnings"] = "hrearn ~ educ + exper + expersq + nrtheast + married + male"

In [3]:
from linearmodels.system import SUR

mod = SUR.from_formula(formula, data)
print(mod.fit(cov_type="unadjusted"))

                           System GLS Estimation Summary                           
Estimator:                        GLS   Overall R-squared:                   0.6951
No. Equations.:                     2   McElroy's R-squared:                 0.2197
No. Observations:                 616   Judge's (OLS) R-squared:             0.1873
Date:                Wed, Aug 17 2022   Berndt's R-squared:                  0.3775
Time:                        08:11:57   Dhrymes's R-squared:                 0.6950
                                        Cov. Estimator:                  unadjusted
                                        Num. Constraints:                      None
                Equation: benefits, Dependent Variable: hrbens                
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
educ           0.0346     0.0050     6.8605     0.0000      0.0247      0.0445
exper       

## Curly Braces {}

The same formula can be expressed in a single string by surrounding each equation with braces `{}`.

In [4]:
braces_formula = """
{hrbens ~ educ + exper + expersq + union + south + nrtheast + nrthcen + male}
{hrearn ~ educ + exper + expersq + nrtheast + married + male}
"""
braces_mod = SUR.from_formula(braces_formula, data)
braces_res = braces_mod.fit(cov_type="unadjusted")
print(braces_res)

                           System GLS Estimation Summary                           
Estimator:                        GLS   Overall R-squared:                   0.6951
No. Equations.:                     2   McElroy's R-squared:                 0.2197
No. Observations:                 616   Judge's (OLS) R-squared:             0.1873
Date:                Wed, Aug 17 2022   Berndt's R-squared:                  0.3775
Time:                        08:11:57   Dhrymes's R-squared:                 0.6950
                                        Cov. Estimator:                  unadjusted
                                        Num. Constraints:                      None
                 Equation: hrbens, Dependent Variable: hrbens                 
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
educ           0.0346     0.0050     6.8605     0.0000      0.0247      0.0445
exper       

## Labeled Formulas

When using the curly brace formula specification, the equation names are determined by the dependent variable names.  When names are repeated as is the case in some datasets (e.g. a SUR on GDP of multiple countries) then the equation labels will be modified until they are unique.  This can produce meaningless equation labels, and so it is possible to pass an equation label using the syntax

```
{label : dep ~ exog}
```


In [5]:
labeled_formula = """
{benefits: hrbens ~ educ + exper + expersq + union + south + nrtheast + nrthcen + male}
{earnings: hrearn ~ educ + exper + expersq + nrtheast + married + male}
"""
labels_mod = SUR.from_formula(labeled_formula, data)
labeled_res = labels_mod.fit(cov_type="unadjusted")

print("Unlabeled")
print(braces_res.equation_labels)
print("Labeled")
print(labeled_res.equation_labels)

Unlabeled
['hrbens', 'hrearn']
Labeled
['benefits', 'earnings']


## Other Options

### Estimation Weights

SUR supports weights which are assumed to be proportional to the inverse variance of the data so that 

$$ V(y_i \times w_i) = \sigma^2 \,\,\forall i.$$

Weights can be passed using a `DataFrame` where each column.  

Here the results are printed to ensure that the estimates are different from those in the standard GLS model.

In [6]:
random_weights = np.random.chisquare(5, size=(616, 2))
random_weights = pd.DataFrame(random_weights, columns=["benefits", "earnings"])
weighted_mod = SUR.from_formula(formula, data, weights=random_weights)
print(weighted_mod.fit())

                           System GLS Estimation Summary                           
Estimator:                        GLS   Overall R-squared:                   0.6835
No. Equations.:                     2   McElroy's R-squared:                 0.2130
No. Observations:                 616   Judge's (OLS) R-squared:             0.1837
Date:                Wed, Aug 17 2022   Berndt's R-squared:                  0.4244
Time:                        08:11:57   Dhrymes's R-squared:                 0.6833
                                        Cov. Estimator:                      robust
                                        Num. Constraints:                      None
                Equation: benefits, Dependent Variable: hrbens                
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
educ           0.0293     0.0052     5.6463     0.0000      0.0192      0.0395
exper       

### Prespecified Residual Covariance
Like a standard SUR, it is possible to pass a prespecified residual covariance for use in the GLS step.  This is done using the keyword argument `sigma` in the `from_formula` method, and is otherwise identical to passing one to the standard SUR.